In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [2]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory("../Keras/dogs-vs-cats/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory("../Keras/dogs-vs-cats/validation", target_size=(224,224))

Found 18750 images belonging to 2 classes.
Found 6250 images belonging to 2 classes.


### ResNet50 Summary

In [4]:
from keras.applications.resnet50 import ResNet50
resnet = ResNet50()
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

### ResNet 50 layer 구현

In [7]:
# Input Layer
from keras import Input
import keras.layers as layers
input_tensor = Input(shape=(224,224,3)) #0
x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_tensor) #1
# Conv1
x = layers.Conv2D(64, 7, strides=(2, 2), padding='valid', name='conv1')(x) #2
# BatchNormalization? : 뉴럴넷을 학습시킬 때 보통 mini-batch 단위로 데이터를 가져와서 
#                       학습을 시키는데, 각 feature 별로 평균과 표준편차를 구해준 다음 
#                       normalize 해주고, scale factor와 shift factor를 이용하여 
#                       새로운 값을 만들어준다.  / axis parameter : 정수형
x = layers.BatchNormalization(name='bn_conv1')(x) #3
x = layers.Activation('relu')(x) #4
x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x) #5

In [ ]:
# Conv2_x
x = layers.MaxPooling2D((3, 3), strides=(2, 2), name='max_pool2d_1')(x) #6
shortcut = x #shortcut = 6(MaxPooling)

x = layers.Conv2D(64, 1, strides=(1, 1), padding='valid',
                  name='res2a_branch2a')(x) #7
x = layers.BatchNormalization(name='bn2a_branch2a')(x) #8
x = layers.Activation('relu')(x) #9
x = layers.Conv2D(64, 3, strides=(1, 1), padding='same',
                  name='res2a_branch2b')(x) #10
x = layers.BatchNormalization(name='bn2a_branch2b')(x) #11
x = layers.Activation('relu')(x) #12
x = layers.Conv2D(256, 1, strides=(1, 1), padding='valid',
                  name='res2a_branch2c')(x) #13
x = layers.Conv2D(256, 1, strides=(1, 1), padding='valid',
                  name='res2a_branch1')(shortcut) #14
x = layers.BatchNormalization(name='bn2a_branch2c')(x) #15
shortcut = layers.BatchNormalization(name='bn2a_branch1')(shortcut) #16 BatchNormalize shortcut
x = layers.add([x,shortcut]) #17 x+shortcut Residual
x = layers.Activation('relu')(x) #18
shortcut = x #shortcut = 18(relu)
